In [1]:
# load data with pandas
import pandas as pd

path_to_data = "data/elife-31098-supp1-v2.xls"
df = pd.read_excel(path_to_data)

In [2]:
# inspect data with head method
df.head()

,Gene,log2 Fold Change (H/L) (KRASG12V/Empty Vector),p-value
0,ABCC1,0.649462,0.015625
1,ABCC3,1.597891,0.250000
2,ACP2,-1.499113,0.500000
3,ACSL3,-1.279971,0.500000
4,ADAM10,-0.065187,1.000000


In [3]:
# load bokeh dependencies for the volcano plot 
from bokeh.plotting import figure
from bokeh.layouts import layout
from bokeh.embed import components
from bokeh.io import show
from bokeh.io import output_notebook

In [4]:
# set up notebok environment to render bokeh plots inline
output_notebook()

Loading BokehJS ...

In [5]:
# create the volcano plot

# we'll need numpy for the log transform
import numpy as np

# initialize figure object
p = figure(plot_width=400, plot_height=400)

# add the points as a circle glyph
p.circle(x=df['log2 Fold Change (H/L) (KRASG12V/Empty Vector)'],
         y=-np.log10(df['p-value']))

# format the axis labels
p.xaxis.axis_label = "log2(H/L) (KRAS-G12V/EV)" #TODO: Include conditions
p.yaxis.axis_label = "-log10(p-value)"

# render the plot
show(p)

In [6]:
# add horizontal and vertical cutoffs using spans 
#(http://bokeh.pydata.org/en/latest/docs/user_guide/annotations.html#spans)

In [7]:
# add colors

In [8]:
# add tooltips

In [9]:
# multiple hypothesis correction

In [10]:
# update tool tips 

In [11]:
# save plot as a vector graphic
p.output_backend = "svg"
p.toolbar.logo = None